In [7]:
from phe import paillier # 开源库
import random # 选择随机数

def shift_encrypt_decrypt(data, key, encrypt = True):
    data_bytes = data.to_bytes((data.bit_length() + 7) // 8, byteorder = 'big')
    key_bytes = key.to_bytes((key.bit_length() + 7) // 8, byteorder = 'big')
    data_bytearray = bytearray(data_bytes)
    key_bytearray = bytearray(key_bytes)
    
    if encrypt:
        for i in range(len(data_bytearray)):
            data_bytearray[i] = (data_bytearray[i] + key_bytearray[i % len(key_bytearray)]) % 256
    else:
        for i in range(len(data_bytearray)):
            data_bytearray[i] = (data_bytearray[i] - key_bytearray[i % len(key_bytearray)]) % 256
    
    return int.from_bytes(data_bytearray, byteorder = 'big')

# key 为加解密密钥
key = 666666

##################### 设置参数
# 服务器端保存的数值
message_list = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
# 服务器要保存的加密后数值
server_list = []
for i in message_list:
    encrypted_data = shift_encrypt_decrypt(i, key, encrypt = True)
    server_list.append(encrypted_data)
    # print(encrypted_data)
# 加密后的数组
print("密钥 k 加密后的数组:", server_list)
length = len(server_list)
# 客户端生成公私钥
public_key, private_key = paillier.generate_paillier_keypair()
# 客户端随机选择一个要读的位置
pos = random.randint(0, length - 1)
print("要读起的数值位置为：", pos)

##################### 客户端生成密文选择向量
select_list = []
enc_list = []
for i in range(length):
    select_list.append(i == pos)
    enc_list.append(public_key.encrypt(select_list[i]))

##################### 服务器端进行运算
c = 0
for i in range(length):
    c = c + server_list[i] * enc_list[i]
print("产生密文：", c.ciphertext())

##################### 客户端进行解密
m = private_key.decrypt(c)
print("得到用密钥 k 加密后的数值:", m)
decrypted_data = shift_encrypt_decrypt(m, key, encrypt = False)
print("得到原始数值：", decrypted_data)


密钥 k 加密后的数组: [110, 210, 2904, 3004, 2848, 3204, 3304, 3404, 3504, 3348]
要读起的数值位置为： 1
产生密文： 756768519579433205786095134242169306976838740435160101453034802340308348789668119009690566729484690158321815542853540955627784605057795728988736564253127954858051044701700693062788899463684688081149184535027897592046597278669087216084138114989816658821904157880670845509629672014653554538097019145930393376456527710865085700190325128632823444418017991543116252424165960645123963067645396112547987313957467026048418394264559897298053391657808258559868763406501942847550624247852879292843635139596076165432740342769096918512123200386002713350531989076333239275701396352589246527079045335766960636075827060864545341852223936761925215384556491330932677912074539021524276290804933950711709462663602349441827713955326762042157412169045304243190083463286330358710447425508169355555154801135974104452531518302603390211095294000424293309183358844593894833651920817455633892204334178595683457569211067170335647194569616